In [5]:
# Importing necessary modules
import os
import time
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score, mean_squared_error
from model_data_definitions import *
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

# Function to generate cross validation dataset
def generate_cross_validation_dataset(data, num_folds):
    X, y = data.drop('target', axis=1).values, data['target'].values
    
    dataset_list = []
    kf = KFold(n_splits=num_folds)
    fold = 0

    # Splitting data into training and testing set for each fold in the cross-validation 
    for train_index, test_index in kf.split(X):
        fold += 1
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        X_train, X_test = preprocess_data(X_train, X_test)
        y_train, y_test = preprocess_target_values(y_train, y_test)
        
        dataset_list.append((X_train, y_train, X_test, y_test , fold))
    
    return dataset_list

# Function to train and evaluate model
def train_evaluate_model(model_tuple, fold_data, epoch_number, dataset_name,num_folds):
    
    
    model, name = model_tuple
    X_train, y_train, X_test, y_test , fold = fold_data
    
    print(f"Evaluating{name} for {fold}")
    
    # Training the model 
    history = model.fit(X_train,
                        y_train,
                        epochs=epoch_number,
                        verbose=1,
                        validation_data=(X_test,y_test))

     # Evaluating the trained model on test data 
    loss = model.evaluate(X_test,y_test)

     # Making predictions on the test data 
    predictions = model.predict(X_test)
    
     # Calculate metrics 
    r_squared_value=r2_score(y_true=y_test,y_pred=predictions)
    test_error=mean_squared_error(y_true=y_test,y_pred=predictions)

    results = {
        'model': name,
        'fold': fold,
        'train_history': history.history['loss'],
        'val_history': history.history['val_loss'],
        'loss': loss,
        'r_squared_value': r_squared_value,
        'test_error': test_error}

    # Save results to numpy file
    if not os.path.exists('aggregate_results'):
        os.makedirs('aggregate_results')

    np.save(f'aggregate_results/{dataset_name}-{name}-epochs-{epoch_number}-fold-{fold}-of-{num_folds}.npy', results)

# Function to evaluate models in parallel
def evaluate_models_parallel(fold_data, dataset_name, epoch_number,num_folds):
    print(f"models_init{fold_data[4]}")
    models = initialize_all_models(fold_data[0].shape[1], seed_val=fold_data[4])
    compile_models(models)

     # Training and evaluating all models in parallel using ThreadPoolExecutor 
    with ThreadPoolExecutor() as executor:
        futures = {executor.submit(train_evaluate_model, 
                                   model, 
                                   fold_data, 
                                   epoch_number, 
                                   dataset_name,
                                   num_folds): model for model in models}
        for future in futures:
            future.result()  # Just to make sure all tasks are finished

# Function to evaluate all folds in parallel
def evaluate_all_folds_parallel(kfold_datasets, dataset_name, epoch_number,num_folds):
    
     # Evaluating all folds in parallel using ThreadPoolExecutor 
    with ThreadPoolExecutor() as executor:
        futures = {executor.submit(evaluate_models_parallel, 
                                   fold_data, 
                                   dataset_name, 
                                   epoch_number,
                                   num_folds): fold_data for fold_data in kfold_datasets}
        for future in futures:
            future.result()  # Just to make sure all tasks are finished

# New function to retrieve all datasets and their names and feed it to relevant functions with a for loop.
def retrieve_datasets_and_run_evaluations(num_folds=5, epoch_number=100):
    # Fetching data 
    filtered_datasets_metadata, datasets = fetch_return_filtered_pmlb_data_sets()

    for dataset, row in zip(datasets, filtered_datasets_metadata.iterrows()):
        dataset_name = row[1]['dataset']
        kfold_datasets = generate_cross_validation_dataset(dataset, num_folds)
        evaluate_all_folds_parallel(kfold_datasets, dataset_name, epoch_number,num_folds)

In [6]:
# Function to evaluate models in parallel
def evaluate_models_parallel(fold_data, dataset_name, epoch_number,num_folds):
    
    models = initialize_all_models(fold_data[0].shape[1], seed_val=fold_data[4])
    compile_models(models)

     # Training and evaluating all models sequentially 
    for model in models:
        train_evaluate_model(model, fold_data, epoch_number, dataset_name,num_folds) 

# Function to evaluate all folds in parallel
def evaluate_all_folds_parallel(kfold_datasets, dataset_name, epoch_number,num_folds):
    
     # Evaluating all folds sequentially 
    for fold_data in kfold_datasets:
        evaluate_models_parallel(fold_data, dataset_name, epoch_number,num_folds)

# New function to retrieve all datasets and their names and feed it to relevant functions with a for loop.
def retrieve_datasets_and_run_evaluations(num_folds=5, epoch_number=100):
    # Fetching data 
    filtered_datasets_metadata, datasets = fetch_return_filtered_pmlb_data_sets()

    for dataset, row in zip(datasets, filtered_datasets_metadata.iterrows()):
        dataset_name = row[1]['dataset']
        kfold_datasets = generate_cross_validation_dataset(dataset, num_folds)
        evaluate_all_folds_parallel(kfold_datasets, dataset_name, epoch_number,num_folds)

In [7]:
filtered_datasets_metadata, datasets = fetch_return_filtered_pmlb_data_sets()
filtered_datasets_metadata

,dataset,n_instances,n_features,n_binary_features,n_categorical_features,n_continuous_features,endpoint_type,n_classes,imbalance,task
0,1027_ESL,488,4,0,0,4,continuous,9.0,0.099363,regression
2,1029_LEV,1000,4,0,0,4,continuous,5.0,0.111245,regression
3,1030_ERA,1000,4,0,0,4,continuous,9.0,0.031251,regression
5,1096_FacultySalaries,50,4,0,0,4,continuous,39.0,0.004063,regression
13,192_vineyard,52,2,0,0,2,continuous,19.0,0.040475,regression
23,228_elusage,55,2,0,0,2,continuous,52.0,0.000953,regression
25,230_machine_cpu,209,6,0,0,6,continuous,116.0,0.004906,regression
29,485_analcatdata_vehicle,48,4,0,0,4,continuous,47.0,0.000434,regression
32,519_vinnie,380,2,0,0,2,continuous,16.0,0.030146,regression
34,523_analcatdata_neavote,100,2,0,0,2,continuous,8.0,0.136914,regression


In [3]:
# Check if kfold_datasets contains valid numbers
for dataset in kfold_datasets:
    X_train, y_train, X_test, y_test , fold = dataset
    if not (np.isfinite(X_train).all() and np.isfinite(y_train).all() and np.isfinite(X_test).all() and np.isfinite(y_test).all()):
        print(f"Dataset for fold {fold} contains invalid numbers.")
    else:
        print(f"Dataset for fold {fold} is valid.")

NameError: name 'kfold_datasets' is not defined

In [8]:
num_folds = 10

for dataset, row in zip(datasets, filtered_datasets_metadata.iterrows()):
    dataset_name = row[1]['dataset']
    if dataset_name == 'titanic':
        print(row)
        #print(dataset)
        kfold_datasets = generate_cross_validation_dataset(dataset, num_folds)
        # Check if kfold_datasets contains valid numbers
        for dataset in kfold_datasets:
            X_train, y_train, X_test, y_test , fold = dataset
            if not (np.isfinite(X_train).all() and np.isfinite(y_train).all() and np.isfinite(X_test).all() and np.isfinite(y_test).all()):
                print(f"Dataset for fold {fold} contains invalid numbers.")
            else:
                print(f"Dataset for fold {fold} is valid.")
        evaluate_all_folds_parallel(kfold_datasets, dataset_name, epoch_number=101,num_folds=10)
        


(270, dataset                      titanic
n_instances                     2201
n_features                         3
n_binary_features                  0
n_categorical_features             0
n_continuous_features              3
endpoint_type             continuous
n_classes                        2.0
imbalance                   0.125266
task                      regression
Name: 270, dtype: object)
Dataset for fold 1 is valid.
Dataset for fold 2 is valid.
Dataset for fold 3 is valid.
Dataset for fold 4 is valid.
Dataset for fold 5 is valid.
Dataset for fold 6 is valid.
Dataset for fold 7 is valid.
Dataset for fold 8 is valid.
Dataset for fold 9 is valid.
Dataset for fold 10 is valid.
EvaluatingLinear Model for 1
Epoch 1/101
60/60 [==============================] - 0s 2ms/step - loss: 0.8021 - val_loss: 0.8101
Epoch 2/101
60/60 [==============================] - 0s 1ms/step - loss: 0.7613 - val_loss: 0.7881
Epoch 3/101
60/60 [==============================] - 0s 1ms/step - loss: 0.7242 

Epoch 76/101
60/60 [==============================] - 0s 1ms/step - loss: 0.4674 - val_loss: 0.5435
Epoch 77/101
60/60 [==============================] - 0s 1ms/step - loss: 0.4673 - val_loss: 0.5434
Epoch 78/101
60/60 [==============================] - 0s 1ms/step - loss: 0.4674 - val_loss: 0.5434
Epoch 79/101
60/60 [==============================] - 0s 1ms/step - loss: 0.4672 - val_loss: 0.5433
Epoch 80/101
60/60 [==============================] - 0s 1ms/step - loss: 0.4673 - val_loss: 0.5435
Epoch 81/101
60/60 [==============================] - 0s 1ms/step - loss: 0.4673 - val_loss: 0.5433
Epoch 82/101
60/60 [==============================] - 0s 1ms/step - loss: 0.4673 - val_loss: 0.5432
Epoch 83/101
60/60 [==============================] - 0s 1ms/step - loss: 0.4673 - val_loss: 0.5434
Epoch 84/101
60/60 [==============================] - 0s 1ms/step - loss: 0.4674 - val_loss: 0.5434
Epoch 85/101
60/60 [==============================] - 0s 1ms/step - loss: 0.4673 - val_loss: 0.5433


60/60 [==============================] - 0s 1ms/step - loss: 0.4614 - val_loss: 0.5440
Epoch 56/101
60/60 [==============================] - 0s 1ms/step - loss: 0.4551 - val_loss: 0.5481
Epoch 57/101
60/60 [==============================] - 0s 1ms/step - loss: 0.4558 - val_loss: 0.5355
Epoch 58/101
60/60 [==============================] - 0s 1ms/step - loss: 0.4531 - val_loss: 0.5374
Epoch 59/101
60/60 [==============================] - 0s 1ms/step - loss: 0.4526 - val_loss: 0.5426
Epoch 60/101
60/60 [==============================] - 0s 1ms/step - loss: 0.4508 - val_loss: 0.5365
Epoch 61/101
60/60 [==============================] - 0s 1ms/step - loss: 0.4566 - val_loss: 0.5521
Epoch 62/101
60/60 [==============================] - 0s 1ms/step - loss: 0.4520 - val_loss: 0.5500
Epoch 63/101
60/60 [==============================] - 0s 1ms/step - loss: 0.4513 - val_loss: 0.5382
Epoch 64/101
60/60 [==============================] - 0s 1ms/step - loss: 0.4503 - val_loss: 0.5436
Epoch 65/101


60/60 [==============================] - 0s 1ms/step - loss: 0.4103 - val_loss: 0.5521
Epoch 35/101
60/60 [==============================] - 0s 1ms/step - loss: 0.4112 - val_loss: 0.5715
Epoch 36/101
60/60 [==============================] - 0s 1ms/step - loss: 0.4166 - val_loss: 0.5479
Epoch 37/101
60/60 [==============================] - 0s 1ms/step - loss: 0.4072 - val_loss: 0.5770
Epoch 38/101
60/60 [==============================] - 0s 1ms/step - loss: 0.4077 - val_loss: 0.5329
Epoch 39/101
60/60 [==============================] - 0s 1ms/step - loss: 0.4052 - val_loss: 0.5217
Epoch 40/101
60/60 [==============================] - 0s 1ms/step - loss: 0.4071 - val_loss: 0.5510
Epoch 41/101
60/60 [==============================] - 0s 1ms/step - loss: 0.4028 - val_loss: 0.5586
Epoch 42/101
60/60 [==============================] - 0s 1ms/step - loss: 0.4039 - val_loss: 0.5597
Epoch 43/101
60/60 [==============================] - 0s 1ms/step - loss: 0.4009 - val_loss: 0.5357
Epoch 44/101


60/60 [==============================] - 0s 1ms/step - loss: 0.6989 - val_loss: 0.8761
Epoch 14/101
60/60 [==============================] - 0s 1ms/step - loss: 0.6815 - val_loss: 0.8710
Epoch 15/101
60/60 [==============================] - 0s 1ms/step - loss: 0.6790 - val_loss: 0.8711
Epoch 16/101
60/60 [==============================] - 0s 1ms/step - loss: 0.6791 - val_loss: 0.8711
Epoch 17/101
60/60 [==============================] - 0s 1ms/step - loss: 0.6791 - val_loss: 0.8710
Epoch 18/101
60/60 [==============================] - 0s 1ms/step - loss: 0.6790 - val_loss: 0.8711
Epoch 19/101
60/60 [==============================] - 0s 1ms/step - loss: 0.6792 - val_loss: 0.8712
Epoch 20/101
60/60 [==============================] - 0s 1ms/step - loss: 0.6791 - val_loss: 0.8711
Epoch 21/101
60/60 [==============================] - 0s 1ms/step - loss: 0.6790 - val_loss: 0.8710
Epoch 22/101
60/60 [==============================] - 0s 1ms/step - loss: 0.6790 - val_loss: 0.8711
Epoch 23/101


...

60/60 [==============================] - 0s 1ms/step - loss: 0.4363 - val_loss: 0.6478
Epoch 92/101
60/60 [==============================] - 0s 1ms/step - loss: 0.4349 - val_loss: 0.6470
Epoch 93/101
60/60 [==============================] - 0s 1ms/step - loss: 0.4334 - val_loss: 0.6459
Epoch 94/101
60/60 [==============================] - 0s 1ms/step - loss: 0.4321 - val_loss: 0.6446
Epoch 95/101
60/60 [==============================] - 0s 1ms/step - loss: 0.4309 - val_loss: 0.6442
Epoch 96/101
60/60 [==============================] - 0s 1ms/step - loss: 0.4297 - val_loss: 0.6433
Epoch 97/101
60/60 [==============================] - 0s 1ms/step - loss: 0.4284 - val_loss: 0.6424
Epoch 98/101
60/60 [==============================] - 0s 1ms/step - loss: 0.4272 - val_loss: 0.6416
Epoch 99/101
60/60 [==============================] - 0s 1ms/step - loss: 0.4262 - val_loss: 0.6410
Epoch 100/101
60/60 [==============================] - 0s 1ms/step - loss: 0.4251 - val_loss: 0.6401
Epoch 101/10

60/60 [==============================] - 0s 2ms/step - loss: 0.4453 - val_loss: 0.5426
Epoch 71/101
60/60 [==============================] - 0s 2ms/step - loss: 0.4452 - val_loss: 0.5407
Epoch 72/101
60/60 [==============================] - 0s 2ms/step - loss: 0.4457 - val_loss: 0.5424
Epoch 73/101
60/60 [==============================] - 0s 2ms/step - loss: 0.4443 - val_loss: 0.5428
Epoch 74/101
60/60 [==============================] - 0s 2ms/step - loss: 0.4461 - val_loss: 0.5404
Epoch 75/101
60/60 [==============================] - 0s 2ms/step - loss: 0.4437 - val_loss: 0.5418
Epoch 76/101
60/60 [==============================] - 0s 2ms/step - loss: 0.4454 - val_loss: 0.5422
Epoch 77/101
60/60 [==============================] - 0s 2ms/step - loss: 0.4457 - val_loss: 0.5411
Epoch 78/101
60/60 [==============================] - 0s 2ms/step - loss: 0.4445 - val_loss: 0.5442
Epoch 79/101
60/60 [==============================] - 0s 2ms/step - loss: 0.4453 - val_loss: 0.5406
Epoch 80/101


60/60 [==============================] - 0s 1ms/step - loss: 0.4699 - val_loss: 0.5627
Epoch 50/101
60/60 [==============================] - 0s 1ms/step - loss: 0.4688 - val_loss: 0.5623
Epoch 51/101
60/60 [==============================] - 0s 1ms/step - loss: 0.4677 - val_loss: 0.5622
Epoch 52/101
60/60 [==============================] - 0s 1ms/step - loss: 0.4673 - val_loss: 0.5606
Epoch 53/101
60/60 [==============================] - 0s 1ms/step - loss: 0.4666 - val_loss: 0.5595
Epoch 54/101
60/60 [==============================] - 0s 1ms/step - loss: 0.4651 - val_loss: 0.5586
Epoch 55/101
60/60 [==============================] - 0s 1ms/step - loss: 0.4643 - val_loss: 0.5586
Epoch 56/101
60/60 [==============================] - 0s 1ms/step - loss: 0.4640 - val_loss: 0.5570
Epoch 57/101
60/60 [==============================] - 0s 1ms/step - loss: 0.4632 - val_loss: 0.5573
Epoch 58/101
60/60 [==============================] - 0s 1ms/step - loss: 0.4628 - val_loss: 0.5557
Epoch 59/101


60/60 [==============================] - 5s 79ms/step - loss: 0.6364 - val_loss: 0.8968
Epoch 29/101
60/60 [==============================] - 5s 79ms/step - loss: 0.6301 - val_loss: 0.8945
Epoch 30/101
60/60 [==============================] - 5s 80ms/step - loss: 0.6241 - val_loss: 0.8920
Epoch 31/101
60/60 [==============================] - 5s 78ms/step - loss: 0.6183 - val_loss: 0.8900
Epoch 32/101
60/60 [==============================] - 5s 80ms/step - loss: 0.6127 - val_loss: 0.8876
Epoch 33/101
60/60 [==============================] - 5s 79ms/step - loss: 0.6076 - val_loss: 0.8855
Epoch 34/101
60/60 [==============================] - 5s 80ms/step - loss: 0.6025 - val_loss: 0.8831
Epoch 35/101
60/60 [==============================] - 5s 78ms/step - loss: 0.5977 - val_loss: 0.8810
Epoch 36/101
60/60 [==============================] - 5s 79ms/step - loss: 0.5929 - val_loss: 0.8786
Epoch 37/101
60/60 [==============================] - 5s 78ms/step - loss: 0.5879 - val_loss: 0.8765
Epo

KeyboardInterrupt: 

SyntaxError: positional argument follows keyword argument (2579379979.py, line 1)

In [ ]:
"""# New function to retrieve all datasets and their names and feed it to relevant functions with a for loop.
def retrieve_datasets_and_run_evaluations_specific(num_folds=10, epoch_number=100):
    # Fetching data 
    filtered_datasets_metadata, datasets = fetch_return_filtered_pmlb_data_sets()

    for dataset, row in zip(datasets, filtered_datasets_metadata.iterrows()):
        dataset_name = row[1]['dataset']
        if dataset_name == 'banana':
            kfold_datasets = generate_cross_validation_dataset(dataset, num_folds)
            evaluate_all_folds_parallel(kfold_datasets, dataset_name, epoch_number,num_folds)
            
retrieve_datasets_and_run_evaluations_specific(num_folds=10, epoch_number=100)
"""

In [ ]:
# Call the new function 
start_time = time.time()
retrieve_datasets_and_run_evaluations(num_folds=10, epoch_number=100)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"The experiment took {elapsed_time} seconds to complete.")

2/2 [==============================] - 1s 477ms/step - loss: 0.2831


2/2 [==============================] - 1s 533ms/step - loss: 0.2313


4/4 [==============================] - 19s 239ms/step


4/4 [==============================] - 9s 2s/step


1/1 [==============================] - 1s 541ms/step - loss: 0.8857


...

1/1 [==============================] - 2s 2s/step - loss: 0.4225


1/1 [==============================] - 5s 5s/step


1/1 [==============================] - 1s 1s/step - loss: 0.4222


17/17 [==============================] - 5s 275ms/step


17/17 [==============================] - 1s 67ms/step


In [ ]:
# Call the new function 
start_time = time.time()
retrieve_datasets_and_run_evaluations(num_folds=5, epoch_number=1)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"The experiment took {elapsed_time} seconds to complete.")

In [ ]:
# Call the new function 
start_time = time.time()
retrieve_datasets_and_run_evaluations(num_folds=10, epoch_number=1)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"The experiment took {elapsed_time} seconds to complete.")

In [ ]:
# Call the new function 
start_time = time.time()
retrieve_datasets_and_run_evaluations(num_folds=10, epoch_number=5)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"The experiment took {elapsed_time} seconds to complete.")

In [ ]:
# Call the new function 
start_time = time.time()
retrieve_datasets_and_run_evaluations(num_folds=10, epoch_number=10)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"The experiment took {elapsed_time} seconds to complete.")

In [ ]:
# Call the new function 
start_time = time.time()
retrieve_datasets_and_run_evaluations(num_folds=10, epoch_number=20)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"The experiment took {elapsed_time} seconds to complete.")

In [ ]:
# Call the new function 
start_time = time.time()
retrieve_datasets_and_run_evaluations(num_folds=10, epoch_number=40)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"The experiment took {elapsed_time} seconds to complete.")

In [ ]:
# Call the new function 
start_time = time.time()
retrieve_datasets_and_run_evaluations(num_folds=10, epoch_number=100)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"The experiment took {elapsed_time} seconds to complete.")